<a href="https://colab.research.google.com/github/Vitor178/TrabSO/blob/master/amd_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# importa macro %%cu
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

# carrega plugin
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-mlhvxixx
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-mlhvxixx
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=b6c7cd0dd289958864263f4a0a025c258fdce8d351ff273ea83b9c6959c51b68
  Stored in directory: /tmp/pip-ephem-wheel-cache-fyeg2wgt/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [47]:
#%%cu
%%writefile ProjFinal.cu

#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>
#include <pthread.h>
#include <math.h>

#define MAX_BLOCK 128
#define MAX_THREADS 32

//First line: v (number of vertices) and e (number of edges)


// Solves the minimum distance between all pairs of vertices

__global__
void md_all_pairs (uint32_t* dists, uint32_t v, uint32_t k) {
    //int _id = (int )(id);
    //long int _id = (long int)(id);

    uint32_t portion_size = (v*v)/(blockDim.x * gridDim.x);
    
    uint32_t i_real = (blockIdx.x * blockDim.x + threadIdx.x)/v;
    uint32_t j_real = (blockIdx.x * blockDim.x + threadIdx.x)%v;

    uint32_t i_final = (blockIdx.x * blockDim.x + threadIdx.x + portion_size)/v;
    uint32_t j_final = (blockIdx.x * blockDim.x + threadIdx.x + portion_size)%v;


    //if (v%(blockDim.x * gridDim.x) != 0)
    //    portion_size++;


    while ((j_real < j_final || i_real < i_final) && j_real < v && i_real < v) {           
      uint32_t intermediary = dists[i_real*v+k] + dists[k*v+j_real];
      // Checks for overflows
      if ((intermediary >= dists[i_real*v+k])&&(intermediary >= dists[k*v+j_real])&&(intermediary < dists[i_real*v+j_real]))
        dists[i_real*v+j_real] = dists[i_real*v+k] + dists[k*v+j_real];
        j_real++;
        if (j_real == v){
            j_real = 0;
            i_real++;
        }
    }
        
}

__global__
void amd_threads (uint32_t* dists, uint32_t v, uint32_t *smd, uint32_t *paths) 
{    
    uint32_t i, j;
	  uint32_t infinity = v*v;
    
    uint32_t portion_size = v/(blockDim.x * gridDim.x);
    uint32_t i_real = (blockIdx.x * blockDim.x + threadIdx.x)/portion_size;
 //printf("bx = %d, bd = %d, td = %d, i_real = %d\n", blockIdx.x, blockDim.x, threadIdx.x, i_real);
    
    //if (v%(blockDim.x * gridDim.x) != 0)        //divisao nao exata
    //    portion_size++;

    for (i = i_real*portion_size; (i < ((i_real+1)* portion_size)) && (i < v); ++i) {
        for (j = 0; j < v; ++j) {
			// We only consider if the vertices are different and there is a path
            if ((i != j) && (dists[i*v+j] < infinity)) {
				smd[i_real] += dists[i*v+j];       //salva 
				paths[i_real]++;
			}
        }
    }
}

/* Computes the average minimum distance between all pairs of vertices with a path connecting them */
void amd (uint32_t* dists, uint32_t v, uint32_t div_t, uint32_t div_b) {
    
    uint32_t *smd; 	//sum of minimum distances
    uint32_t *paths; //number of paths found
   

    cudaMallocManaged(&smd, (div_t * div_b) * sizeof(uint32_t));
    cudaMallocManaged(&paths, (div_t * div_b) * sizeof(uint32_t));
    for (int i = 0; i < (div_t * div_b); i++){
        smd[i] = 0;
        paths[i] = 0;
    }

	uint32_t solution = 0, smd_sum = 0, paths_sum = 0;
    
    amd_threads<<<div_b, div_t>>>(dists, v, smd, paths);
    //printf("Entrou no amd\n");

	  cudaDeviceSynchronize();
    //printf("chamou amd_threads\n");

    for (int i = 0; i < (div_t * div_b); i++){
        smd_sum += smd[i];
        paths_sum += paths[i];
        //printf("%d,%d",smd[i],paths[i]);
    }
  for (int k = 0; k<v;k++){
      for (int j = 0;j<v;j++){
          printf(" %d ",dists[k*v+j]);
      }
      printf("\n");
  }
  fflush(stdout);
  if (paths_sum)  //impedi divisao por 0
	  solution = smd_sum / paths_sum;
	printf("media = %d\n", solution);
}

/* Debug function (not to be used when measuring performance)*/
void debug (uint32_t* dists, uint32_t v) {
    uint32_t i, j;
	uint32_t infinity = v*v;
    

    for (i = 0; i < v; ++i) {
        for (j = 0; j < v; ++j) {
            if (dists[i*v+j] > infinity) printf("%7s", "inf");
            else printf ("%7u", dists[i*v+j]);
        }
        printf("\n");
    }
}

// Main program - reads input, calls FW, shows output
int main (int argc, char* argv[]) {
    uint32_t v, e;

    //printf("ola");
    fflush(stdout);
    //Reads input 
    scanf("%u %u", &v, &e);
    //printf("%d,%d\n",v,e);

    uint32_t *dists;

    cudaMallocManaged(&dists, v*v*sizeof(uint32_t));

    //Allocates distances matrix (w/ size v*v) i
    //and sets it with max distance and 0 for own vertex
    memset(dists, UINT32_MAX, v*v*sizeof(uint32_t));
    uint32_t i;
    for ( i = 0; i < v; ++i ) dists[i*v+i] = 0;

    //Reads edges from file and sets them in the distance matrix
    uint32_t source, dest, cost;
    for ( i = 0; i < e; ++i ){
        scanf("%u %u %u", &source, &dest, &cost);
        if (cost < dists[source*v+dest]) dists[source*v+dest] = cost;
    }

    //printf("Terminou Scan\n");
   //calcula a divisao para enviar pra gpu
   uint32_t div_t = v*v, div_b = 1;
   while (div_t > MAX_THREADS){   //tem que ficar abaixo do limite de threads
       div_t /= 2;
       div_b *= 2;
   }
   if (div_b > MAX_BLOCK){
      div_b = MAX_BLOCK;
   }

    //printf("Trata os tamanhos\n");
   //chama as threads
  for (int k = 0;k<v; k++){
	  //Computes the minimum distance for all pairs of vertices using k as shortcut
    md_all_pairs<<<div_b, div_t>>>(dists, v, k);

    //espera as threads acabarem
    cudaDeviceSynchronize();
  }
    //printf("Executou md all pairs\n");


    //md_all_pairs(dists, v);

    //divisao aqui ocorre apenas em v
   div_t = v, div_b = 1;
   while (div_t > MAX_THREADS){   //tem que ficar abaixo do limite de threads
       div_t /= 2;
       div_b *= 2;
   }
   if (div_b > MAX_BLOCK){
      div_b = MAX_BLOCK;
   }
    //Computes and prints the final solution
    amd(dists, v, div_t, div_b);

	// Free memory
  cudaFree(dists);

  printf("acabou");
#if DEBUG
	debug(dists, v);
#endif

    return 0;
}

Overwriting ProjFinal.cu


In [48]:
!nvcc ProjFinal.cu -o proj && ./proj

33 18 0 4 7 0 2 1 2 4 2 4 7 4 7 9 2 4 9 15 4 8 6 7 0 7 7 4 1 8 3 8 9 3 2 5 2 7  6 4 1 1 4 7 11 2 5 13 8 2 15 19 30 18 2 7
 0  -1  1  11  3  -1  -1  7  9  9  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1 
 18  0  19  15  7  -1  -1  11  13  13  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1 
 13  -1  0  10  2  -1  -1  6  8  8  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1 
 -1  -1  -1  0  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1 
 11  -1  12  8  0  -1  -1  4  6  6  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1 
 20  -1  7  17  9  0  -1  13  15  15  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1 
 12  -1  13  9  1  -1  0  5  7  7  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1 